# Confluence + LLM = QA

In [1]:
%pip install pandas --quiet
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


In [2]:
from os import environ
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

confluence_token = environ.get('CONFLUENCE_TOKEN')
hf_token = environ.get('HF_TOKEN')
hf_write_token = environ.get('HF_WRITE_TOKEN')
gigachat_token = environ.get('GIGACHAT_TOKEN')

In [3]:
from atlassian import Confluence

confluence_url = "https://confluence.utmn.ru"
confluence = Confluence(url=confluence_url, token=confluence_token)

## Поиск документа в Confluence через CQL

 * https://atlassian-python-api.readthedocs.io/confluence.html
 * https://developer.atlassian.com/cloud/confluence/rest/v2/api-group-page/#api-pages-id-get
 * https://developer.atlassian.com/server/confluence/advanced-searching-using-cql/
 * https://spacy.io/usage/spacy-101

In [8]:
!python -m spacy download ru_core_news_sm --quiet

✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [4]:
from bs4 import BeautifulSoup
from langchain_community.document_loaders import PyPDFLoader

import spacy
nlp = spacy.load("ru_core_news_sm")
needed_pos = ['NOUN', 'NUM', 'PROPN', 'ADJ', 'VERB', 'X']


def get_cql_query(spaces, question):
    exclude = ' and label != "навигация"' 
    words = [token for token in nlp(question.lower()) if not token.is_stop and
             token.pos_ in needed_pos and len(token.text) > 2]
    if len(words) == 0:
        return ()
    spaces = " or ".join([f"space = {space}" for space in spaces])
    words_with_verbs = " and ".join(list(set([f"(title ~ '{word}*' or text ~ '{word}*' or title ~ '{word.lemma_}*' or text ~ '{word.lemma_}*')"
                                              for word in words])))
    words_without_verbs = " and ".join(list(set([f"(title ~ '{word}*' or text ~ '{word}*' or title ~ '{word.lemma_}*' or text ~ '{word.lemma_}*')"
                                                 for word in words if word.pos_ != 'VERB'])))
    words_without_verbs_and_adj = " and ".join(list(set([f"(title ~ '{word}*' or text ~ '{word}*' or title ~ '{word.lemma_}*' or text ~ '{word.lemma_}*')"
                                                         for word in words if word.pos_ not in ['VERB', 'ADJ']])))
    return ("(" + spaces + ") and (" + words_with_verbs + ")" + exclude,
            "(" + spaces + ") and (" + words_without_verbs + ")" + exclude,
            "(" + spaces + ") and (" + words_without_verbs_and_adj + ")" + exclude)
    
    
def get_document_id(question: str) -> str:
    cql_query = get_cql_query(spaces=["study"], question=question)
    if len(cql_query) == 0:
        return "0"
    results = confluence.cql(cql_query[0], start=0, limit=1)['results']
    if len(results) == 0:
        results = confluence.cql(cql_query[1], start=0, limit=1)['results']
        if len(results) == 0:
            results = confluence.cql(cql_query[2], start=0, limit=1)['results']
            if len(results) == 0:
                return "0"

    return results[0]['content']['id']


def get_document_content_by_id(page_id: str):
    page = confluence.get_page_by_id(page_id, expand='space,body.export_view')
    page_body = page['body']['export_view']['value']
    page_download = page['_links']['base'] + page['_links']['download'] if 'download' in page['_links'].keys() else ''

    try:
        if len(page_body) > 50:
            page_body = page['body']['export_view']['value']
            soup = BeautifulSoup(page_body, 'html.parser')
            page_body_text = soup.get_text(separator=' ')
            content = page_body_text.replace(" \n ", "")
        elif '.pdf' in page_download.lower():
            loader = PyPDFLoader(page_download.split('?')[0])
            content = " ".join([page.page_content for page in loader.load_and_split()])
        else:
            return None
    except:
        return None

    return content


def get_document_content(question: str):
    page_id = get_document_id(question)
    if page_id == "0":
        return None
    return get_document_content_by_id(page_id)

In [11]:
%%timeit
get_document_content("Как поменять физкультуру?")

216 ms ± 20.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Собственный индекс Confluence

### Структура БД

In [4]:
from sqlalchemy import create_engine
engine = create_engine(f"postgresql://{environ.get('POSTGRES_USER')}:{environ.get('POSTGRES_PASSWORD')}@{environ.get('POSTGRES_HOST')}/{environ.get('POSTGRES_DB')}", echo=False)

In [12]:
from typing import Optional
from pgvector.sqlalchemy import Vector
from sqlalchemy import Text, select, text
from sqlalchemy.orm import DeclarativeBase, Mapped, Session, mapped_column

class Base(DeclarativeBase):
    pass

class Document(Base):
    __tablename__ = "document"
    id: Mapped[int] = mapped_column(primary_key=True)
    confluence_id: Mapped[int] = mapped_column(index=True)
    text: Mapped[str] = mapped_column(Text())
    text_lem: Mapped[str] = mapped_column(Text())
    tfidf: Mapped[Optional[Vector]] = mapped_column(Vector(2936))
    doc2vec: Mapped[Optional[Vector]] = mapped_column(Vector(150))
    rubert: Mapped[Optional[Vector]] = mapped_column(Vector(312))
    rusbert: Mapped[Optional[Vector]] = mapped_column(Vector(312))
    rusbert_finetuned: Mapped[Optional[Vector]] = mapped_column(Vector(312))
    

In [17]:
def lower_stopword_lemmatize(text):
    return " ".join([token.lemma_ for token in nlp(str(text).lower()) if not token.is_stop and token.pos_ != 'PUNCT'])

### Выгрузка, предобработка и сохранение документов из пространства

In [8]:
pages = confluence.cql("space = study and label != \"навигация\"", start=0, limit=100)['results']
page_ids = [page['content']['id'] for page in pages if 'content' in page.keys()]
conf_df = pd.DataFrame({"page_id": page_ids})
conf_df["content"] = conf_df["page_id"].apply(get_document_content_by_id)
conf_df

,page_id,content
0,86479083,Сопровождение\nспециализированных\nобразовател...
1,86479057,2 \n \n \n \n1. ОБЩИЕ ПОЛОЖЕНИЯ \n \n1.1. По...
2,86479066,MuHucrepcreo HayKu 14 BbtcuJero o6paaoeaHnR Po...
3,86479079,Об утверждении Регламента \nпроведения промежу...
4,86479078,Об утверждении Р егламента \nпроведения промеж...
5,86479077,MrHucrepcreo HayKn I Bbtctuero o6pasoeaHnR poc...
6,86479076,Об утверждении Регламента \nпроведения промежу...
7,86479075,Об утверждении Регламента \nпроведения промежу...
8,86479074,MrHncrepcreo HayKu 14 Bbtcuero o6paroeaHnn poc...
9,86479073,MrHucrepcreo HayKu t4 Bbtcuero o6pa:oeaHnR Poc...


In [9]:
trash_id = ["86479082", "86479093", "86479069", "86479070", "86479071", "86479072", "86479073", "86479074", "86479077", "86479066"]
conf_df["page_id"] = conf_df["page_id"].apply(lambda x: None if x in trash_id else x)
conf_df = conf_df.dropna().reset_index(drop=True)
conf_df.head(5)

,page_id,content
0,86479083,Сопровождение\nспециализированных\nобразовател...
1,86479057,2 \n \n \n \n1. ОБЩИЕ ПОЛОЖЕНИЯ \n \n1.1. По...
2,86479079,Об утверждении Регламента \nпроведения промежу...
3,86479078,Об утверждении Р егламента \nпроведения промеж...
4,86479076,Об утверждении Регламента \nпроведения промежу...


In [10]:
with Session(engine) as session:
    session.execute(text('CREATE EXTENSION IF NOT EXISTS vector'))
    session.commit()
Base.metadata.create_all(engine)

2024-02-14 11:50:51,556 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-02-14 11:50:51,557 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-14 11:50:51,589 INFO sqlalchemy.engine.Engine select current_schema()
2024-02-14 11:50:51,590 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-14 11:50:51,605 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-02-14 11:50:51,606 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-14 11:50:51,610 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-14 11:50:51,612 INFO sqlalchemy.engine.Engine CREATE EXTENSION IF NOT EXISTS vector
2024-02-14 11:50:51,613 INFO sqlalchemy.engine.Engine [generated in 0.00098s] {}
2024-02-14 11:50:51,642 INFO sqlalchemy.engine.Engine COMMIT
2024-02-14 11:50:51,645 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-14 11:50:51,649 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid =

In [13]:
with Session(engine) as session:
    for index, row in conf_df.iterrows():
        doc = Document(
            confluence_id=int(row["page_id"]),
            text=row["content"], 
            text_lem=lower_stopword_lemmatize(row["content"])
        )
        session.add(doc)
    session.commit()   

2024-02-14 11:53:03,757 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-14 11:53:03,757 INFO sqlalchemy.engine.Engine INSERT INTO document (confluence_id, text, text_lem, tfidf, doc2vec, rubert, rusbert, rusbert_finetuned) SELECT p0::INTEGER, p1::TEXT, p2::TEXT, p3::VECTOR(2936), p4::VECTOR(150), p5::VECTOR(312), p6::VECTOR(312), p7::VECTOR(312) FROM (VALUES (%(conf ... 5737 characters truncated ... 2, p3, p4, p5, p6, p7, sen_counter) ORDER BY sen_counter RETURNING document.id, document.id AS id__1
2024-02-14 11:53:03,757 INFO sqlalchemy.engine.Engine [generated in 0.00031s (insertmanyvalues) 1/1 (ordered)] {'tfidf__0': None, 'rusbert__0': None, 'rusbert_finetuned__0': None, 'doc2vec__0': None, 'text__0': 'Сопровождение\nспециализированных\nобразовательных треков Спорт.\nпрограмми -\nрование\nТреки\nКапитаныШкола \nестественных\nнаукИнтеграция Нетология ... (3774 characters truncated) ... я 46 студентовСпорт.\nпрограмми -\nрование\nЧисленность треков\n57 студентовКапитаны\n80 ст

### Эмбеддинги

In [13]:
with Session(engine) as session:
   db_documents_lem = pd.Series([doc.text_lem for doc in session.scalars(select(Document).order_by(Document.id)).all()])
db_documents_lem.head()

0    сопровождение \n специализированный \n образов...
1    2 \n \n \n \n 1 общий положение  \n \n 1.1   п...
2    утверждение регламент \n проведение промежуточ...
3    утверждение егламента \n проведение промежуточ...
4    утверждение регламент \n проведение промежуточ...
dtype: object

#### TFIDF

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidfX = tfidf_vectorizer.fit_transform(db_documents_lem)
len(tfidf_vectorizer.transform([db_documents_lem[0]]).toarray()[0])

2936

In [19]:
with Session(engine) as session:
   documents = session.scalars(select(Document).order_by(Document.id)).all()
   for doc in documents:
      doc.tfidf = tfidf_vectorizer.transform([doc.text_lem]).toarray()[0]
      session.add(doc)
      session.flush()
   session.commit()

In [18]:
def answer_tfidf(question):
    with Session(engine) as session:
        return session.scalars(select(Document)
                        .order_by(Document.tfidf.cosine_distance(
                            tfidf_vectorizer.transform([lower_stopword_lemmatize(question)]).toarray()[0]
                            )).limit(1)).first().text

In [19]:
%%timeit
answer_tfidf("Как поменять физкультуру?")

13.6 ms ± 210 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Doc2vec

In [29]:
%pip install gensim --quiet

Note: you may need to restart the kernel to use updated packages.


In [20]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate([text.split() for text in db_documents_lem])]
doc2vec_model = Doc2Vec(documents, vector_size=150, window=5, min_count=1, workers=4)

len(doc2vec_model.infer_vector("мама мыла раму".split()))

150

In [22]:
with Session(engine) as session:
   documents = session.scalars(select(Document).order_by(Document.id)).all()
   for doc in documents:
      doc.doc2vec = doc2vec_model.infer_vector(doc.text_lem.split())
      session.add(doc)
      session.flush()
   session.commit()

In [21]:
def answer_doc2vec(question):
    with Session(engine) as session:
        return session.scalars(select(Document)
                        .order_by(Document.doc2vec.cosine_distance(
                           doc2vec_model.infer_vector(lower_stopword_lemmatize(question).split())
                            )).limit(1)).first().text

In [22]:
%%timeit
answer_doc2vec("Как поменять физкультуру?")

11.7 ms ± 158 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### RuBERT-Tiny

In [28]:
import torch
from transformers import AutoTokenizer, AutoModel
rubert_tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
rubert_model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
rubert_model.cpu()
# rubert_model.cuda()  # uncomment it if you have a GPU

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

len(embed_bert_cls("мама мыла раму", rubert_model, rubert_tokenizer))

312

In [25]:
with Session(engine) as session:
   documents = session.scalars(select(Document).order_by(Document.id)).all()
   for doc in documents:
      doc.rubert = embed_bert_cls(doc.text, rubert_model, rubert_tokenizer)
      session.add(doc)
      session.flush()
   session.commit()

In [29]:
def answer_rubert(question):
    with Session(engine) as session:
        return session.scalars(select(Document)
                        .order_by(Document.rubert.cosine_distance(
                            embed_bert_cls(question, rubert_model, rubert_tokenizer)
                            )).limit(1)).first().text

In [30]:
%%timeit
answer_rubert("Как поменять физкультуру?")

13.2 ms ± 201 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### RuSBERT-Tiny

In [31]:
from sentence_transformers import SentenceTransformer

rusbert_model = SentenceTransformer('cointegrated/rubert-tiny2', device="cpu")
len(rusbert_model.encode("мама мыла раму"))

312

In [28]:
with Session(engine) as session:
   documents = session.scalars(select(Document).order_by(Document.id)).all()
   for doc in documents:
      doc.rusbert = rusbert_model.encode(doc.text)
      session.add(doc)
      session.flush()
   session.commit()

In [32]:
def answer_rusbert(question):
    with Session(engine) as session:
        return session.scalars(select(Document)
                        .order_by(Document.rusbert.cosine_distance(
                            rusbert_model.encode(question)
                            )).limit(1)).first().text

In [34]:
%%timeit
answer_rusbert("Как поменять физкультуру?")

14.3 ms ± 487 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Выбор нужного фрагмента через векторный индекс

### Тестовая выборка с вопроосами

In [ ]:
%pip install datasets --quiet

In [41]:
# TODO: use huggingface datasets
# test_questions = pd.read_csv("test_questions.csv", index_col=0)
test_questions

,question,benchmark
0,где почитать отзывы на элективы?,"Прежде, чем выбрать элективы, рекомендуем почи..."
1,когда можно поменять элективы?,"Прежде, чем выбрать элективы, рекомендуем почи..."
2,как выбрать электив?,"Прежде, чем выбрать элективы, рекомендуем почи..."
3,"что делать, если военкомат просит справку?",Вам необходимо обратиться в Отдел мобилизацион...
4,где посмотреть отзывы на элективы?,"Прежде, чем выбрать элективы, рекомендуем почи..."
...,...,...
78,Как продлить академ?,Для рассмотрения возможности продления академи...
79,"Меня призвали в армию, что делать?",Заявление подается через личный кабинет на пор...
80,Как получить отпуск по уходу заребёнком?,Заявление подается через личный кабинет на пор...
81,Как получить отпуск по мед. показаниям?,Заявление подается через личный кабинет на пор...


In [43]:
test_questions["CQL"] = test_questions["question"].apply(get_document_content)
test_questions["tfidf"] = test_questions["question"].apply(answer_tfidf)
test_questions["doc2vec"] = test_questions["question"].apply(answer_doc2vec)
test_questions["rubert"] = test_questions["question"].apply(answer_rubert)
test_questions["rusbert"] = test_questions["question"].apply(answer_rusbert)
test_questions

,question,benchmark,CQL,tfidf,doc2vec,rubert,rusbert
0,где почитать отзывы на элективы?,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...",Вам необходимо обратиться в Отдел мобилизацион...,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи..."
1,когда можно поменять элективы?,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...",Вам необходимо обратиться в Отдел мобилизацион...,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи..."
2,как выбрать электив?,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...",Заявление подается через личный кабинет на по...,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи..."
3,"что делать, если военкомат просит справку?",Вам необходимо обратиться в Отдел мобилизацион...,Вам необходимо обратиться в Отдел мобилизацион...,Для оформления академической справки (справки ...,"По вопросам оплаты обучения, суммы задолженнос...","При восстановлении на договорное место, после ...","При восстановлении на договорное место, после ..."
4,где посмотреть отзывы на элективы?,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...",Вам необходимо обратиться в Отдел мобилизацион...,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи..."
...,...,...,...,...,...,...,...
78,Как продлить академ?,Для рассмотрения возможности продления академи...,Для рассмотрения возможности продления академи...,Для восстановления студенческого билета Вам ...,Заявление подается через личный кабинет на пор...,"Прежде, чем выбрать элективы, рекомендуем почи...",Заявления на восстановление в ТюмГУ принимаютс...
79,"Меня призвали в армию, что делать?",Заявление подается через личный кабинет на пор...,Заявление подается через личный кабинет на пор...,Документ подписан простой электронной подписью...,"Прежде, чем выбрать элективы, рекомендуем почи...",Заявление подается через личный кабинет на пор...,Вам необходимо обратиться в Отдел мобилизацион...
80,Как получить отпуск по уходу заребёнком?,Заявление подается через личный кабинет на пор...,None,Заявление на академический отпуск подается чер...,Заявление на академический отпуск подается чер...,Заявление подается через личный кабинет на пор...,Заявления на восстановление в ТюмГУ принимаютс...
81,Как получить отпуск по мед. показаниям?,Заявление подается через личный кабинет на пор...,Заявление подается через личный кабинет на пор...,Заявление на академический отпуск подается чер...,"Прежде, чем выбрать элективы, рекомендуем почи...",Заявление подается через личный кабинет на пор...,Заявление подается через личный кабинет на пор...


### Выбор лучшего алгоритма

#### Accuracy

In [44]:
for column in test_questions.columns[1:]:
    print(column, sum(test_questions[column].apply(lambda x: "" if x is None else x) == test_questions.benchmark) / len(test_questions.benchmark))

benchmark 1.0
CQL 0.4578313253012048
tfidf 0.46987951807228917
doc2vec 0.03614457831325301
rubert 0.20481927710843373
rusbert 0.2891566265060241


#### ROUGE-L

In [34]:
%pip install rouge --quiet

Note: you may need to restart the kernel to use updated packages.


In [45]:
from rouge import Rouge
rouge = Rouge()

for column in test_questions.columns[1:]:
    print(column, rouge.get_scores(test_questions[column].apply(lambda x: "-" if x is None else x), test_questions["benchmark"], avg=True)['rouge-l'])

benchmark {'r': 1.0, 'p': 1.0, 'f': 0.9999999950000001}
CQL {'r': 0.5632143844363949, 'p': 0.5845958347701568, 'f': 0.48431107770497556}
tfidf {'r': 0.5880011523079155, 'p': 0.5447905152227257, 'f': 0.5374472424797634}
doc2vec {'r': 0.16050843958847527, 'p': 0.1410953841140969, 'f': 0.11908277460390129}
rubert {'r': 0.2924755513104418, 'p': 0.3693253910367398, 'f': 0.310519861004474}
rusbert {'r': 0.3987787912421958, 'p': 0.42272824555538446, 'f': 0.3842241067180766}


## SBERT Fine Tuning

 * https://www.sbert.net/docs/training/overview.html
 * https://huggingface.co/blog/how-to-train-sentence-transformers

In [3]:
from sentence_transformers import SentenceTransformer

### Генерация обучающей выборки через GigaChat

In [47]:
with Session(engine) as session:
   db_documents = [doc.text for doc in session.scalars(select(Document).order_by(Document.id)).all() if len(doc.text) < 3000]
len(db_documents)

2024-02-13 16:21:43,438 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-13 16:21:43,439 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.text_lem, document.tfidf, document.doc2vec, document.rubert, document.rusbert, document.rusbert_finetuned 
FROM document ORDER BY document.id
2024-02-13 16:21:43,440 INFO sqlalchemy.engine.Engine [cached since 182.4s ago] {}
2024-02-13 16:21:43,502 INFO sqlalchemy.engine.Engine ROLLBACK


28

In [48]:
from langchain.prompts import PromptTemplate

prompt_template = """
Сделай глубокий вдох и действуй как студент. На какие 5 вопросов ты можешь получить ответы из документа в тройных кавычках? Используй разговорный стиль речи и студенческую лексику.

\"\"\"
{content}
\"\"\"

Вопросы:
"""

prompt = PromptTemplate.from_template(prompt_template)

In [49]:
from langchain.llms import GigaChat
giga = GigaChat(credentials=gigachat_token, verify_ssl_certs=False)
giga_chain = prompt | giga
giga_chain.invoke({"content": db_documents[0]}).strip().split("\n")

['1. Когда начинается выбор спортивных секций по Физической культуре?',
 '2. Какие ограничения существуют при записи на спортивные секции?',
 '3. Как проверить наличие конфликтов в расписании спортивных секций и своем расписании?',
 '4. Как изменить выбор спортивных секций?',
 '5. Что произойдет, если студент пропустит два занятия подряд?']

In [50]:
gigachat_docs = []
for doc in db_documents:
    query = {"content": doc}
    giga_questions = giga_chain.invoke(query).strip().split("\n")
    for q in giga_questions:
        gigachat_docs.append({
            "question": q[3:],
            "document": doc
        })
    print(giga_questions)
gigachat_docs = pd.DataFrame(gigachat_docs)
gigachat_docs

['1. Когда начинается выбор спортивных секций по Физической культуре?', '2. Какие ограничения существуют при записи на спортивные секции?', '3. Как проверить наличие конфликтов в расписании спортивных секций и своем расписании?', '4. Как изменить выбор спортивных секций?', '5. Что произойдет, если студент пропустит два занятия подряд?']
['1. Что такое "Отзывус"?', '2. Как можно оставить отзыв на "Отзывусе"?', '3. Где можно найти информацию о том, как поменять электив?', '4. Какая информация отправляется на корпоративную почту в первую учебную неделю семестра?', '5. Какие элективы доступны в 2023 году?']
['1. Какие способы оплаты обучения доступны?', '2. Как связаться с отделом платных образовательных услуг?', '3. Где находится отдел платных образовательных услуг?', '4. Как получить документы для оплаты за обучение материнским капиталом?', '5. Каковы сроки оплаты по договору для студентов разных форм обучения?']
['1. Где можно оформить справку о подтверждении обучения?', '2. Какие терми

,question,document
0,Когда начинается выбор спортивных секций по Фи...,Выбор спортивных секций по Физической культуре...
1,Какие ограничения существуют при записи на спо...,Выбор спортивных секций по Физической культуре...
2,Как проверить наличие конфликтов в расписании ...,Выбор спортивных секций по Физической культуре...
3,Как изменить выбор спортивных секций?,Выбор спортивных секций по Физической культуре...
4,"Что произойдет, если студент пропустит два зан...",Выбор спортивных секций по Физической культуре...
...,...,...
153,Как подать заявление на отчисление переводом?,Подать заявление на отчисление переводом можно...
154,Где находится Единый деканат?,Подать заявление на отчисление переводом можно...
155,Какие документы нужно приложить к заявлению на...,Подать заявление на отчисление переводом можно...
156,Сколько времени занимает подготовка приказа об...,Подать заявление на отчисление переводом можно...


In [51]:
# TODO: replace with huggingface datasets uploading
gigachat_docs.to_csv("gigachat_docs.csv")

### Тонкая настройка

In [6]:
# TODO: use huggingface datasets
# gigachat_docs = pd.read_csv("gigachat_docs.csv", index_col=0).reset_index(drop=True)
gigachat_docs

,question,document
0,Когда начинается выбор спортивных секций по Фи...,Выбор спортивных секций по Физической культуре...
1,Какие ограничения существуют при записи на спо...,Выбор спортивных секций по Физической культуре...
2,Как проверить наличие конфликтов в расписании ...,Выбор спортивных секций по Физической культуре...
3,Как изменить выбор спортивных секций?,Выбор спортивных секций по Физической культуре...
4,"Что произойдет, если студент пропустит два зан...",Выбор спортивных секций по Физической культуре...
...,...,...
135,Как подать заявление на отчисление переводом?,Подать заявление на отчисление переводом можно...
136,Где находится Единый деканат?,Подать заявление на отчисление переводом можно...
137,Какие документы нужно приложить к заявлению на...,Подать заявление на отчисление переводом можно...
138,Сколько времени занимает подготовка приказа об...,Подать заявление на отчисление переводом можно...


In [7]:
import math
from sentence_transformers import InputExample, losses
from torch.utils.data import DataLoader

train_set = []
for index, row in gigachat_docs.iterrows():
    train_set.append(InputExample(texts=[row['question'], row['document']]))
    
finetuned_model = SentenceTransformer("cointegrated/rubert-tiny2", device="cuda")

train_dataloader = DataLoader(train_set, shuffle=True, batch_size=16)
# train_loss = losses.MegaBatchMarginLoss(finetuned_model)
train_loss = losses.MegaBatchMarginLoss(finetuned_model)

num_epochs = 10
warmup_steps = math.ceil(len(train_set) * num_epochs * 0.1)

finetuned_model.fit(train_objectives=[(train_dataloader, train_loss)], 
                    epochs=num_epochs, 
                    warmup_steps=warmup_steps,
                    output_path="saved_models/rubert-tiny2-wikiutmn-gigachat-qa")


d:\git\virtassist\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Epoch: 100%|██████████| 10/10 [00:17<00:00,  1.76s/it]


### Индексация

In [4]:
finetuned_model = SentenceTransformer('saved_models/rubert-tiny2-wikiutmn-gigachat-qa', device="cpu")
finetuned_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 2048, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 312, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
)

In [17]:
with Session(engine) as session:
   documents = session.scalars(select(Document).order_by(Document.id)).all()
   for doc in documents:
      doc.rusbert_finetuned = finetuned_model.encode(doc.text)
      session.add(doc)
      session.flush()
   session.commit()

In [47]:
def answer_rusbert_finetuned(question):
    with Session(engine) as session:
        return session.scalars(select(Document)
                        .order_by(Document.rusbert_finetuned.cosine_distance(
                            finetuned_model.encode(question)
                            )).limit(1)).first().text

In [48]:
%%timeit
answer_rusbert_finetuned("Как поменять физкультуру?")

14.3 ms ± 606 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Метрики

In [49]:
test_questions["rusbert_finetuned"] = test_questions["question"].apply(answer_rusbert_finetuned)
test_questions

,question,benchmark,CQL,tfidf,doc2vec,rubert,rusbert,rusbert_finetuned
0,где почитать отзывы на элективы?,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...",Вам необходимо обратиться в Отдел мобилизацион...,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи..."
1,когда можно поменять элективы?,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...",Вам необходимо обратиться в Отдел мобилизацион...,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи..."
2,как выбрать электив?,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...",Заявление подается через личный кабинет на по...,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи..."
3,"что делать, если военкомат просит справку?",Вам необходимо обратиться в Отдел мобилизацион...,Вам необходимо обратиться в Отдел мобилизацион...,Для оформления академической справки (справки ...,"По вопросам оплаты обучения, суммы задолженнос...","При восстановлении на договорное место, после ...","При восстановлении на договорное место, после ...",Для оформления академической справки (справки ...
4,где посмотреть отзывы на элективы?,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...",Вам необходимо обратиться в Отдел мобилизацион...,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи..."
...,...,...,...,...,...,...,...,...
78,Как продлить академ?,Для рассмотрения возможности продления академи...,Для рассмотрения возможности продления академи...,Для восстановления студенческого билета Вам ...,Заявление подается через личный кабинет на пор...,"Прежде, чем выбрать элективы, рекомендуем почи...",Заявления на восстановление в ТюмГУ принимаютс...,Заявления на восстановление в ТюмГУ принимаютс...
79,"Меня призвали в армию, что делать?",Заявление подается через личный кабинет на пор...,Заявление подается через личный кабинет на пор...,Документ подписан простой электронной подписью...,"Прежде, чем выбрать элективы, рекомендуем почи...",Заявление подается через личный кабинет на пор...,Вам необходимо обратиться в Отдел мобилизацион...,Вам необходимо обратиться в Отдел мобилизацион...
80,Как получить отпуск по уходу заребёнком?,Заявление подается через личный кабинет на пор...,None,Заявление на академический отпуск подается чер...,Заявление на академический отпуск подается чер...,Заявление подается через личный кабинет на пор...,Заявления на восстановление в ТюмГУ принимаютс...,Заявление подается через личный кабинет на пор...
81,Как получить отпуск по мед. показаниям?,Заявление подается через личный кабинет на пор...,Заявление подается через личный кабинет на пор...,Заявление на академический отпуск подается чер...,"Прежде, чем выбрать элективы, рекомендуем почи...",Заявление подается через личный кабинет на пор...,Заявление подается через личный кабинет на пор...,Заявление подается через личный кабинет на пор...


#### Accuracy

In [50]:
print("rusbert_finetuned", sum(test_questions["rusbert_finetuned"].apply(lambda x: "" if x is None else x) == test_questions.benchmark) / len(test_questions.benchmark))

rusbert_finetuned 0.4939759036144578


#### ROUGE-L

In [51]:
from rouge import Rouge
rouge = Rouge()

print("rusbert_finetuned", rouge.get_scores(test_questions["rusbert_finetuned"].apply(lambda x: "-" if x is None else x), test_questions["benchmark"], avg=True)['rouge-l'])

rusbert_finetuned {'r': 0.6045191332788015, 'p': 0.5805203079635032, 'f': 0.5653750306531498}


### Save to Hub

In [7]:
finetuned_model.save_to_hub(repo_id="nizamovtimur/rubert-tiny2-wikiutmn", token=hf_write_token, train_datasets=["nizamovtimur/wikiutmn-study-gigachat"])

model.safetensors:   0%|          | 0.00/117M [00:00<?, ?B/s]

'https://huggingface.co/nizamovtimur/rubert-tiny2-wikiutmn/commit/b662cf5318bbdb76a25200b0af13cc06768faace'

## Большие языковые модели

https://python.langchain.com/docs/use_cases/question_answering/

In [6]:
from langchain.prompts import PromptTemplate

prompt_template = """
Используй следующий текст в тройных кавычках, чтобы кратко ответить на вопрос студента в конце. 
Не изменяй и не убирай ссылки, адреса и телефоны. Если ты не можешь найти ответ, напиши, что ответ не найден.
Ответ не должен превышать 100 слов.

\"\"\"
{context}
\"\"\"

Вопрос: {question}
"""

prompt = PromptTemplate.from_template(prompt_template)

### GigaChat

In [7]:
from langchain.llms import GigaChat
giga = GigaChat(credentials=gigachat_token, verify_ssl_certs=False)
giga_chain = prompt | giga

In [14]:
# question = questions.question[89]
question = "Я хожу в фитнес-клуб. Как заменить физкультуру?"
print(question, end="\n\n")
document = answer_rusbert_finetuned(question)
print(document)

Я хожу в фитнес-клуб. Как заменить физкультуру?

Выбор спортивных секций по Физической культуре будет проходить в ИС Модеус во вкладке "Выбор модулей".  Вам нужно будет выбрать 2 интересующие Вас спортивные секции, которые будут проходить каждую неделю в одно и то же время.  Ограничения: 1) Записаться можно не более чем на 2 занятия в неделю 2) Нельзя записываться на два занятия подряд.  Вас могут не допустить на занятие, если Вы были на предыдущей паре и/или уже посетили два занятия за неделю. При этом расписание на наличие конфликтов Вы проверяете самостоятельно в соответствии с Вашим расписанием в ИС Модеус и расписанием спортивных секций (во вложенных файлах). Ваш выбор пролонгируется до конца семестра, однако в любой момент Вы можете его изменить, отписавшись от одной секции и записавшись на другую. ВАЖНО!  Студент, пропустивший два занятия подряд, будет отписан автоматически. Выбор Физической культуры откроется 06.09.2023 и будет открыт до конца семестра.  Для успешной аттестации

In [15]:
query = {"context": document,
        "question": question}
print(giga_chain.invoke(query).strip())
print()
print(query)

Для замены физкультуры на посещение спортивного зала необходимо представить тренеру-преподавателю по спорту (куратору по физкультуре) пакет документов, указанных в Положении о реализации дисциплин по физической культуре и спорту. Все документы должны быть предоставлены в течение первой учебной недели.

{'context': 'Выбор спортивных секций по Физической культуре будет проходить в ИС Модеус во вкладке "Выбор модулей".\xa0 Вам нужно будет выбрать 2 интересующие Вас спортивные секции, которые будут проходить каждую неделю в одно и то же время.\xa0 Ограничения: 1) Записаться можно не более чем на 2 занятия в неделю 2) Нельзя записываться на два занятия подряд.\xa0 Вас могут не допустить на занятие, если Вы были на предыдущей паре и/или уже посетили два занятия за неделю. При этом расписание на наличие конфликтов Вы проверяете самостоятельно в соответствии с Вашим расписанием в ИС Модеус и расписанием спортивных секций (во вложенных файлах). Ваш выбор пролонгируется до конца семестра, однако

### Локальные Text2Text

In [16]:
import torch
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

device = torch.cuda.current_device() if torch.cuda.is_available() and torch.cuda.mem_get_info()[0] >= 2*1024**3 else -1
device

0

In [26]:
# FOR SAVING MODEL ONLY!
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# model_name = "IlyaGusev/fred_t5_ru_turbo_alpaca"
# model_name = "ai-forever/FRED-T5-large"
# model_name = "ai-forever/FRED-T5-1.7B"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# pipe = pipeline(
#     "text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=500
# )
# pipe.save_pretrained("saved_models/FRED-T5-1.7B")

tokenizer_config.json: 100%|██████████| 20.2k/20.2k [00:00<00:00, 15.1MB/s]
d:\git\virtassist\.venv\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Timur\.cache\huggingface\hub\models--ai-forever--FRED-T5-1.7B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
vocab.json: 100%|██████████| 1.71M/1.71M 

In [17]:
saved_pipeline = pipeline("text2text-generation", "saved_models/FRED-T5-1.7B", device=device, max_new_tokens=10000)
hf_model = HuggingFacePipeline(pipeline=saved_pipeline).bind(stop=["\n\n"])
gpu_chain = prompt | hf_model

Loading checkpoint shards: 100%|██████████| 2/2 [01:35<00:00, 47.53s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
query = {"context": document,
        "question": question}
print(gpu_chain.invoke(query).strip())
print()
print(query)

Ответ:
В соответствии с приказом ректора № 555 от 20.08.2023 «О реализации дополнительных образовательных программ в рамках реализации ФГОС ВО» в университете реализуются дополнительные образовательные программы в рамках реализации ФГОС ВО.  В соответствии с приказом ректора № 555 от 20.08.2023 «О реализации дополнительных образовательных программ в рамках реализации ФГОС ВО» в университете реализуются дополнительные образовательные программы в рамках реализации ФГОС ВО.  В соответствии с приказом ректора № 555 от 20.08.2023 «О реализации дополнительных образовательных программ в рамках реализации ФГОС ВО» в университете реализуются дополнительные образовательные программы в рамках реализации ФГОС ВО.  В соответствии с приказом ректора № 555 от 20.08.2023 «О реализации дополнительных образовательных программ в рамках реализации ФГОС ВО» в университете реализуются дополнительные образовательные программы в рамках реализации ФГОС ВО.  В соответствии с приказом ректора № 555 от 20.08.2023

#### Гигиена

In [20]:
del gpu_chain
del hf_model
del saved_pipeline
torch.cuda.empty_cache()